In [14]:
import os
import pandas as pd
import numpy as np
import scipy.io
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Conv2D, MaxPooling2D, Flatten, Dense, GlobalAveragePooling2D, 
    Reshape, multiply, add, Activation, Lambda, Concatenate
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

print("TensorFlow Version:", tf.__version__)

TensorFlow Version: 2.16.2


In [15]:
# --- > UPDATE THIS PATH < ---
# Point this to the main project directory containing the bearing type folders
DATASET_PATH = '/Users/Nitin/Downloads/DDMM'

# Model Hyperparameters
IMG_SIZE = 128
BATCH_SIZE = 32
EPOCHS = 30

In [16]:
def parse_filenames_to_dataframe(dataset_path):
    """
    Walks through all bearing type, sampling rate, and speed subdirectories
    to create a single, comprehensive DataFrame.
    """
    records = []
    expanded_path = os.path.expanduser(dataset_path)
    print(f"Scanning main directory: {expanded_path}...")

    if not os.path.isdir(expanded_path):
        print(f"Error: Directory not found. Check DATASET_PATH.")
        return pd.DataFrame()

    # Find the bearing type subdirectories automatically
    bearing_dirs = [d for d in os.listdir(expanded_path) if os.path.isdir(os.path.join(expanded_path, d))]
    print(f"Found bearing type folders: {bearing_dirs}")

    for bearing_type in bearing_dirs:
        bearing_path = os.path.join(expanded_path, bearing_type)
        for root, _, files in os.walk(bearing_path):
            for file in files:
                if file.endswith('.mat'):
                    # Extract info from filepath and filename
                    path_parts = root.split(os.sep)
                    sampling_rate_str = next((part for part in path_parts if 'SamplingRate' in part), None)
                    
                    file_parts = file.replace('.mat', '').split('_')
                    if len(file_parts) == 5 and sampling_rate_str:
                        rotating_fault, bearing_fault, _, _, speed = file_parts
                        sampling_rate = int(sampling_rate_str.split('_')[-1])
                        
                        records.append({
                            'filepath': os.path.join(root, file),
                            'filename': file,
                            'bearing_type': bearing_type,
                            'sampling_rate': sampling_rate,
                            'speed': int(speed)
                        })
    
    df = pd.DataFrame(records)
    print(f"Scan complete. Found {len(df)} total .mat files across all datasets.")
    return df

In [17]:
master_df = parse_filenames_to_dataframe(DATASET_PATH)
if not master_df.empty:
    display(master_df.head())

Scanning main directory: /Users/Nitin/Downloads/DDMM...
Found bearing type folders: ['DeepGroveBall', 'CylindricalRollerBearing)', '.git', 'TaperedRollerBearing']
Scan complete. Found 1149 total .mat files across all datasets.


,filepath,filename,bearing_type,sampling_rate,speed
0,/Users/Nitin/Downloads/DDMM/DeepGroveBall/Bear...,U3_B_8_6204_600.mat,DeepGroveBall,8000,600
1,/Users/Nitin/Downloads/DDMM/DeepGroveBall/Bear...,M3_H_8_6204_600.mat,DeepGroveBall,8000,600
2,/Users/Nitin/Downloads/DDMM/DeepGroveBall/Bear...,M2_OR_8_6204_600.mat,DeepGroveBall,8000,600
3,/Users/Nitin/Downloads/DDMM/DeepGroveBall/Bear...,M1_IR_8_6204_600.mat,DeepGroveBall,8000,600
4,/Users/Nitin/Downloads/DDMM/DeepGroveBall/Bear...,M2_B_8_6204_600.mat,DeepGroveBall,8000,600


In [18]:
def process_labels(df):
    """Creates binary task labels and domain (speed) labels."""
    print("\nProcessing labels for the full dataset...")
    
    # 1. Binary Task Label (Healthy vs. Faulty)
    df['binary_label'] = df['filename'].apply(lambda x: 0 if x.startswith('H_H_') else 1)
    
    # 2. Domain Label (for speed)
    df['domain_label'], domain_classes = pd.factorize(df['speed'])

    print("Label processing complete.")
    print("\nBinary Label Distribution:\n", df['binary_label'].value_counts())
    print(f"\nFound {len(domain_classes)} unique domains (speeds):", sorted(list(domain_classes)))
    
    return df, list(domain_classes)

In [19]:
if not master_df.empty:
    master_df, domain_classes = process_labels(master_df)
    display(master_df.sample(n=5, random_state=42)) # Show a random sample


Processing labels for the full dataset...
Label processing complete.

Binary Label Distribution:
 binary_label
1    1113
0      36
Name: count, dtype: int64

Found 6 unique domains (speeds): [600, 800, 1000, 1200, 1400, 1600]


,filepath,filename,bearing_type,sampling_rate,speed,binary_label,domain_label
174,/Users/Nitin/Downloads/DDMM/DeepGroveBall/Bear...,M1_H_8_6204_1200.mat,DeepGroveBall,8000,1200,1,5
752,/Users/Nitin/Downloads/DDMM/CylindricalRollerB...,U3_IR_16_NJ204_1200.mat,CylindricalRollerBearing),16000,1200,1,5
467,/Users/Nitin/Downloads/DDMM/CylindricalRollerB...,H_IR_8_NJ204_1000.mat,CylindricalRollerBearing),8000,1000,1,2
240,/Users/Nitin/Downloads/DDMM/DeepGroveBall/Bear...,L_OR_16_6204_1400.mat,DeepGroveBall,16000,1400,1,1
667,/Users/Nitin/Downloads/DDMM/CylindricalRollerB...,M2_OR_16_N204_1000.mat,CylindricalRollerBearing),16000,1000,1,2


In [20]:
def load_and_process_file(filepath, binary_label, domain_label, num_domain_classes):
    """Loads a single .mat file and processes its 78 spectrograms."""
    mat_contents = scipy.io.loadmat(filepath.numpy())
    spectrogram_data = mat_contents['Spectrogram']
    
    if np.max(spectrogram_data) > 0:
        spectrogram_data = spectrogram_data / np.max(spectrogram_data)
    images = np.expand_dims(spectrogram_data, axis=-1)
    
    binary_labels = np.array([binary_label] * 78, dtype=np.float32)
    domain_labels_cat = to_categorical([domain_label] * 78, num_classes=num_domain_classes)
    
    return images, binary_labels, domain_labels_cat

def tf_load_and_process(filepath, binary_label, domain_label, num_domain_classes):
    """A TensorFlow wrapper for the loading function."""
    images, bin_labels, dom_labels = tf.py_function(
        func=load_and_process_file,
        inp=[filepath, binary_label, domain_label, num_domain_classes],
        Tout=(tf.float32, tf.float32, tf.float32)
    )
    
    images.set_shape([78, IMG_SIZE, IMG_SIZE, 1])
    bin_labels.set_shape([78])
    dom_labels.set_shape([78, num_domain_classes])
    
    labels_dict = {"task_output": bin_labels, "domain_output": dom_labels}
    return tf.data.Dataset.from_tensor_slices((images, labels_dict))

In [21]:
if not master_df.empty:
    num_domain_classes = len(domain_classes)
    train_df, val_df = train_test_split(master_df, test_size=0.2, random_state=42)

    # --- Build the Training Pipeline ---
    train_dataset = tf.data.Dataset.from_tensor_slices((
        train_df['filepath'].values,
        train_df['binary_label'].values,
        train_df['domain_label'].values
    ))
    train_dataset = train_dataset.shuffle(buffer_size=len(train_df))
    train_dataset = train_dataset.interleave(
        lambda fp, bl, dl: tf_load_and_process(fp, bl, dl, num_domain_classes),
        cycle_length=tf.data.AUTOTUNE, num_parallel_calls=tf.data.AUTOTUNE
    )
    train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

    # --- Build the Validation Pipeline ---
    val_dataset = tf.data.Dataset.from_tensor_slices((
        val_df['filepath'].values,
        val_df['binary_label'].values,
        val_df['domain_label'].values
    ))
    val_dataset = val_dataset.interleave(lambda fp, bl, dl: tf_load_and_process(fp, bl, dl, num_domain_classes))
    val_dataset = val_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

    print("\n✅ Full tf.data pipelines created successfully for the complete dataset.")


✅ Full tf.data pipelines created successfully for the complete dataset.


In [ ]:
@tf.custom_gradient
def grad_reverse(x):
    y = tf.identity(x)
    def custom_grad(dy):
        return -dy
    return y, custom_grad

class GradientReversal(tf.keras.layers.Layer):
    """Custom layer to reverse the gradient during backpropagation."""
    def __init__(self):
        super().__init__()

    def call(self, x):
        return grad_reverse(x)

def cbam_module(x, ratio=8):
    """Creates a Convolutional Block Attention Module (CBAM)."""
    # Channel Attention
    channels = x.shape[-1]
    avg_pool = GlobalAveragePooling2D()(x)
    max_pool = tf.keras.layers.GlobalMaxPooling2D()(x)
    
    shared_dense_one = Dense(channels // ratio, activation='relu')
    shared_dense_two = Dense(channels)
    
    avg_out = shared_dense_two(shared_dense_one(avg_pool))
    max_out = shared_dense_two(shared_dense_one(max_pool))
    
    channel_attention = Activation('sigmoid')(add([avg_out, max_out]))
    channel_attention = Reshape((1, 1, channels))(channel_attention)
    x_channel = multiply([x, channel_attention])

    # Spatial Attention
    avg_pool = Lambda(lambda y: tf.reduce_mean(y, axis=3, keepdims=True))(x_channel)
    max_pool = Lambda(lambda y: tf.reduce_max(y, axis=3, keepdims=True))(x_channel)
    
    concat = Concatenate(axis=3)([avg_pool, max_pool])
    spatial_attention = Conv2D(1, (7, 7), padding='same', activation='sigmoid')(concat)
    
    return multiply([x_channel, spatial_attention])


In [22]:
def build_full_model(input_shape, num_domain_classes):
    """Builds the Domain-Adaptive CNN with CBAM."""
    inputs = Input(shape=input_shape)

    # Step 2: CNN Backbone
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)

    # Step 3: Attention Module
    attention_output = cbam_module(x)

    x = Conv2D(128, (3, 3), activation='relu', padding='same')(attention_output)
    features = MaxPooling2D((2, 2))(x)
    flattened_features = Flatten()(features)

    # Branch 1: Main Task (Binary Classification)
    task_head = Dense(128, activation='relu')(flattened_features)
    task_output = Dense(1, activation='sigmoid', name='task_output')(task_head)

    # Branch 2: Domain Adaptation
    grl = GradientReversal()(flattened_features)
    domain_head = Dense(128, activation='relu')(grl)
    domain_output = Dense(num_domain_classes, activation='softmax', name='domain_output')(domain_head)
    
    model = Model(inputs=inputs, outputs=[task_output, domain_output])
    return model

In [23]:
if 'master_df' in locals():
    model = build_full_model(
        input_shape=(IMG_SIZE, IMG_SIZE, 1), 
        num_domain_classes=num_domain_classes
    )

    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss={
            'task_output': 'binary_crossentropy',
            'domain_output': 'categorical_crossentropy'
        },
        loss_weights={'task_output': 1.0, 'domain_output': 0.5},
        metrics={'task_output': 'accuracy'}
    )
    
    print("\n--- Full Model Summary ---")
    model.summary()


--- Full Model Summary ---


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 128, 128,  │        320 │ input_layer_1[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_3     │ (None, 64, 64,    │          0 │ conv2d_4[0][0]    │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 64, 64,    │     18,496 │ max_pooling2d_3[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_4     │ (None, 32, 32,    │          0 │ conv2d_5[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 64)        │          0 │ max_pooling2d_4[… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 64)        │          0 │ max_pooling2d_4[… │
│ (GlobalMaxPooling2… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 8)         │        520 │ global_average_p… │
│                     │                   │            │ global_max_pooli… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 64)        │        576 │ dense_4[0][0],    │
│                     │                   │            │ dense_4[1][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 64)        │          0 │ dense_5[0][0],    │
│                     │                   │            │ dense_5[1][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 64)        │          0 │ add_1[0][0]       │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_1 (Reshape) │ (None, 1, 1, 64)  │          0 │ activation_1[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply_2          │ (None, 32, 32,    │          0 │ max_pooling2d_4[… │
│ (Multiply)          │ 64)               │            │ reshape_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_2 (Lambda)   │ (None, 32, 32, 1) │          0 │ multiply_2[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_3 (Lambda)   │ (None, 32, 32, 1) │          0 │ multiply_2[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 32, 32, 2) │          0 │ lambda_2[0][0],   │
│ (Concatenate)       │                   │            │ lambda_3[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 32, 32, 1) │         99 │ concatenate_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply_3          │ (None, 32, 32,    │          0 │ multiply_2[0][0], │
│ (Multiply)          │ 64)               │            │ conv2d_6[0][0]  

 Total params: 8,483,634 (32.36 MB)

 Trainable params: 8,483,634 (32.36 MB)

 Non-trainable params: 0 (0.00 B)

In [24]:
if 'model' in locals():
    print("\n--- Starting Full Model Training on the Complete Dataset ---")
    history = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=EPOCHS,
        verbose=1
    )
    print("\n--- Model Training Complete ---")


--- Starting Full Model Training on the Complete Dataset ---
Epoch 1/30
   2241/Unknown 981s 436ms/step - domain_output_loss: 56.2259 - loss: 51.0478 - task_output_accuracy: 0.9555 - task_output_loss: 22.9349

2025-09-30 14:50:40.594730: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/Users/Nitin/Library/Python/3.9/lib/python/site-packages/keras/src/trainers/epoch_iterator.py:160: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


2241/2241 ━━━━━━━━━━━━━━━━━━━━ 1066s 474ms/step - domain_output_loss: 56.2096 - loss: 51.0325 - task_output_accuracy: 0.9555 - task_output_loss: 22.9277 - val_domain_output_loss: 1.7957 - val_loss: 1.1295 - val_task_output_accuracy: 0.9652 - val_task_output_loss: 0.2296
Epoch 2/30


2025-09-30 14:52:05.387469: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


2241/2241 ━━━━━━━━━━━━━━━━━━━━ 0s 478ms/step - domain_output_loss: 1.7934 - loss: 1.0863 - task_output_accuracy: 0.9722 - task_output_loss: 0.1896

...

2025-09-30 23:59:42.797958: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


2241/2241 ━━━━━━━━━━━━━━━━━━━━ 0s 454ms/step - domain_output_loss: 1.7915 - loss: 1.0241 - task_output_accuracy: 0.9719 - task_output_loss: 0.1284

2025-10-01 00:16:41.447636: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


2241/2241 ━━━━━━━━━━━━━━━━━━━━ 1098s 489ms/step - domain_output_loss: 1.7915 - loss: 1.0241 - task_output_accuracy: 0.9719 - task_output_loss: 0.1284 - val_domain_output_loss: 1.7924 - val_loss: 1.0492 - val_task_output_accuracy: 0.9652 - val_task_output_loss: 0.1511

--- Model Training Complete ---


2025-10-01 00:18:01.251356: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


In [25]:
if 'history' in locals():
    print("\n--- Evaluating Final Model on the Complete Validation Set ---")
    results = model.evaluate(val_dataset)
    
    print("\n--- Final Performance ---")
    print(f"Total Validation Loss: {results[0]:.4f}")
    print(f"Binary Task (Fault/Healthy) Validation Loss: {results[1]:.4f}")
    print(f"Domain (Speed) Validation Loss: {results[2]:.4f}")
    print(f"Binary Task (Fault/Healthy) Validation Accuracy: {results[3]*100:.2f}%")


--- Evaluating Final Model on the Complete Validation Set ---
561/561 ━━━━━━━━━━━━━━━━━━━━ 76s 134ms/step - domain_output_loss: 1.7964 - loss: 1.0549 - task_output_accuracy: 0.9637 - task_output_loss: 0.1567

--- Final Performance ---
Total Validation Loss: 1.0492
Binary Task (Fault/Healthy) Validation Loss: 0.1511
Domain (Speed) Validation Loss: 1.7924
Binary Task (Fault/Healthy) Validation Accuracy: 96.52%


2025-10-01 00:20:28.848379: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
